In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'aa22da52-0c5f-482b-8272-a62c9277ca97',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

**Annotations:**

**Imports:**
- `requests` library is used to make HTTP requests to the CoinMarketCap API.
- `json` is used to parse the API response.

**API Configuration:**
- `url`: Endpoint for fetching the latest cryptocurrency listings.
- `parameters`: Specifies the query parameters (`start=1`, `limit=15`, `convert=USD`).
- `headers`: Includes the API key for authentication (`X-CMC_PRO_API_KEY`).

**API Request:**
- A `Session` is created, and headers are updated.
- The `session.get()` method fetches data from the API.
- The response is parsed into a Python dictionary (`json.loads`).
- Errors (e.g., connection issues) are caught and printed.

**Output:**
- The raw JSON response is printed, showing data for the top 15 cryptocurrencies.

In [ ]:
type(data)

**Annotations:**
- Checks the type of the `data` variable.
- Output confirms `data` is a Python `dict` (dictionary), which is the parsed JSON response.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

**Annotations:**
- Imports the `pandas` library for data manipulation.
- Configures pandas to display all columns (`max_columns=None`) and rows (`max_rows=None`) in DataFrames.

In [ ]:
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

**Annotations:**

**Data Normalization:**
- `pd.json_normalize` flattens the nested JSON structure (`data['data']`) into a tabular format (`DataFrame`).
- Columns include cryptocurrency attributes like `id`, `name`, `price`, `market_cap`, etc.

**Timestamp:**
- Adds a `timestamp` column with the current time to track when the data was fetched.

**Output:**
- Displays the DataFrame with all columns and rows (due to earlier pandas configuration).

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'aa22da52-0c5f-482b-8272-a62c9277ca97',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df = pd.concat([df, df2])  # Using concat instead of append
    
    path = 'crypto_data.csv'
    if not os.path.isfile(path):
        df.to_csv(path, header='column_names')
    else:
        df.to_csv(path, mode='a', header=False)

**Annotations:**

**Function Purpose:**
- Automates API calls and appends new data to a CSV file.

**Steps:**
- Repeats the API request logic from earlier.
- Normalizes the new data into `df2` and adds a timestamp.
- Appends `df2` to the global `df` DataFrame.

**Saves df to CSV:**
- Creates the file with headers if it doesn’t exist.
- Appends data without headers if the file exists.

**Note:**
- Uses `global df` to modify the DataFrame outside the function.
- Uses `pd.concat` (modern replacement for deprecated `df.append`).

In [ ]:
import os 
from time import sleep

for i in range(3):  # Reduced to 3 iterations for demo purposes
    api_runner()
    print(f'API Runner completed iteration {i+1}')
    sleep(60)

**Annotations:**

**Imports:**
- `os` for file path operations.
- `sleep` to add delays between API calls.

**Loop:**
- Calls `api_runner()` 3 times (to fetch data periodically).
- Waits 60 seconds between each call.
- Prints a completion message after each run.

## 🔍 Additional Analysis

Let’s now delve into the intricate crucible of percentage changes across various timeframes for each cryptocurrency. This will reveal trends that intertwine short-term volatility with long-term momentum.

In [ ]:
# Calculate mean percentage changes for each cryptocurrency
df3 = df.groupby('name', sort=False)[[
    'quote.USD.percent_change_1h',
    'quote.USD.percent_change_24h',
    'quote.USD.percent_change_7d',
    'quote.USD.percent_change_30d',
    'quote.USD.percent_change_60d',
    'quote.USD.percent_change_90d'
]].mean()

print("\nAverage percentage changes by cryptocurrency:")
print(df3)

In [ ]:
# Reshape the data
df4 = df3.stack()
print("\nStacked percentage changes:")
print(df4)
print(f"\nType of df4: {type(df4)}")

# Convert to DataFrame
df5 = df4.to_frame(name='values')
print("\nConverted to DataFrame:")
print(df5)
print(f"\nCount of values: {df5.count()}")

In [ ]:
# Reset index
df6 = df5.reset_index()
print("\nData with reset index:")
print(df6)

# Rename for clarity
df7 = df6.rename(columns={'level_1': 'percent_change'})

# Clean labels
df7['percent_change'] = df7['percent_change'].replace([
    'quote.USD.percent_change_1h',
    'quote.USD.percent_change_24h',
    'quote.USD.percent_change_7d',
    'quote.USD.percent_change_30d',
    'quote.USD.percent_change_60d',
    'quote.USD.percent_change_90d'
], ['1h', '24h', '7d', '30d', '60d', '90d'])

print("\nCleaned time period labels:")
print(df7)

In [ ]:
#Visualize Percentage Changes
print("\nCreating point plot of percentage changes...")
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point', height=6, aspect=2)
plt.title('Average Percentage Price Changes by Cryptocurrency')
plt.ylabel('Percentage Change')
plt.xlabel('Time Period')
plt.show()


## 📈 Bitcoin’s Price Timeline

Now, we narrow our focus and examine the verdant path of Bitcoin’s price history through time.

In [ ]:
df8 = df[['name', 'quote.USD.price', 'timestamp']]
df8 = df8.query("name == 'Bitcoin'")
print("\nBitcoin price data over time:")
print(df8.head())

In [ ]:
print("\nCreating Bitcoin price timeline...")
sns.set_theme(style="darkgrid")
plt.figure(figsize=(12, 6))
sns.lineplot(x='timestamp', y='quote.USD.price', data=df8)
plt.title('Bitcoin Price Over Time')
plt.ylabel('Price (USD)')
plt.xlabel('Timestamp')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 🧵 Final Thoughts

From tangled threads of momentary fluctuations to the broader tapestry of market trends, we’ve embarked on an intricate voyage across crypto price dynamics.

May your next steps through this ever-shifting mosaic of digital victuals be informed, strategic, and certainly captivating.